# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-03-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-02-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-07-03 04:33:54,34.223334,-82.461707,119,0,0,119,"Abbeville, South Carolina, US",485.179598,0.000000
1,22001.0,Acadia,Louisiana,US,2020-07-03 04:33:54,30.295065,-92.414197,945,37,0,908,"Acadia, Louisiana, US",1523.088081,3.915344
2,51001.0,Accomack,Virginia,US,2020-07-03 04:33:54,37.767072,-75.632346,1042,14,0,1028,"Accomack, Virginia, US",3224.408962,1.343570
3,16001.0,Ada,Idaho,US,2020-07-03 04:33:54,43.452658,-116.241552,2336,23,0,2313,"Ada, Idaho, US",485.062927,0.984589
4,19001.0,Adair,Iowa,US,2020-07-03 04:33:54,41.330756,-94.471059,15,0,0,15,"Adair, Iowa, US",209.731544,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,29481,29640,30175,30451,30616,30967,31238,31517,31836,32022
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,2047,2114,2192,2269,2330,2402,2466,2535,2580,2662
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,12076,12248,12445,12685,12968,13273,13571,13907,14272,14657


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,618,639,675,683,703,721,733,746,774,807
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,45,47,49,51,53,55,58,62,65,69
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,861,869,878,885,892,897,905,912,920,928


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,9260,9869,10174,10306,10674,12604,13934,14131,15651,16041
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1195,1217,1250,1298,1346,1384,1438,1459,1516,1559
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,8674,8792,8920,9066,9202,9371,9674,9897,10040,10342


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,225,226,231,247,247,247,253,257,267,280
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,30,30,30,30,30,30,30,30,30,31


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
1213,1067.0,Henry,Alabama,US,2020-07-03 04:33:54,31.511480,-85.242679,130,3,0,127,"Henry, Alabama, US",755.594304,2.307692
431,1017.0,Chambers,Alabama,US,2020-07-03 04:33:54,32.913601,-85.390727,600,27,0,573,"Chambers, Alabama, US",1804.294220,4.500000
2519,1117.0,Shelby,Alabama,US,2020-07-03 04:33:54,33.268798,-86.662326,1109,23,0,1086,"Shelby, Alabama, US",509.411948,2.073940


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,2739879,128740,781970
Brazil,1496858,61884,957692
Russia,660231,9668,428276


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,2739879,128740,781970,1829169,2020-07-03 04:33:54,38.291091,-91.616688
Brazil,1496858,61884,957692,477282,2020-07-03 04:33:54,-12.669522,-48.480493
Russia,660231,9668,428276,222287,2020-07-03 04:33:54,54.546312,62.120860
India,625544,18213,379892,227439,2020-07-03 04:33:54,23.356511,82.081064
Peru,292004,10045,182097,99862,2020-07-03 04:33:54,-10.637324,-75.277744


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
New York,394954,32064,0
California,246550,6265,0
Texas,179137,2542,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
New York,394954,32064,0,362890,2020-07-03 04:33:54,42.671593,-75.579694
California,246550,6265,0,240285,2020-07-03 04:33:54,37.778248,-120.728666
Texas,179137,2542,0,176595,2020-07-03 04:33:54,31.657778,-98.589862
New Jersey,172356,15107,0,157249,2020-07-03 04:33:54,40.286441,-74.622450
Florida,169106,3617,0,165489,2020-07-03 04:33:54,28.940755,-82.700744


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,215902,23123,0
California,Los Angeles,107792,3457,0
Illinois,Cook,90911,4581,0
Arizona,Maricopa,54757,843,0
New York,Nassau,41910,2184,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,215902,23123,0,192779,2020-07-03 04:33:54,40.767273,-73.971526,36061.0
California,Los Angeles,107792,3457,0,104335,2020-07-03 04:33:54,34.308284,-118.228241,6037.0
Illinois,Cook,90911,4581,0,86330,2020-07-03 04:33:54,41.841448,-87.816588,17031.0
Arizona,Maricopa,54757,843,0,53914,2020-07-03 04:33:54,33.348359,-112.491815,4013.0
New York,Nassau,41910,2184,0,39726,2020-07-03 04:33:54,40.740665,-73.589419,36059.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-30,31517,2535,13907,855,284,69,64530,25542,7920,17766,...,314160,936,8503,5832,355,2428,10,1158,1594,591
2020-07-01,31836,2580,14272,855,291,69,67197,26065,8001,17873,...,314992,943,8781,6062,355,2758,10,1190,1632,605
2020-07-02,32022,2662,14657,855,315,69,69941,26658,8066,17941,...,285268,947,9078,6273,355,3080,10,1221,1632,617


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-30,746,62,912,52,13,3,1307,443,104,705,...,43815,27,26,51,0,8,1,312,24,7
2020-07-01,774,65,920,52,15,3,1351,453,104,705,...,43991,28,26,54,0,8,1,318,30,7
2020-07-02,807,69,928,52,17,3,1385,459,104,705,...,44080,28,27,57,0,9,1,325,30,7


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-30,14131,1459,9897,799,93,22,22028,14048,7040,16478,...,1370,824,5682,1649,335,451,8,488,1329,162
2020-07-01,15651,1516,10040,799,97,22,23040,14563,7090,16491,...,1372,825,5847,1649,336,460,8,504,1348,166
2020-07-02,16041,1559,10342,800,97,23,24186,15036,7130,16514,...,1373,828,6034,2100,340,460,8,513,1348,173


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20
3147,84080001,US,USA,840,80001.0,Out of AL,Alabama,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
52,84001095,US,USA,840,1095.0,Marshall,Alabama,US,34.369760,-86.304867,...,1140,1182,1265,1326,1389,1408,1453,1495,1537,1622
51,84001093,US,USA,840,1093.0,Marion,Alabama,US,34.136974,-87.887042,...,171,179,183,184,188,189,197,200,203,206
50,84001091,US,USA,840,1091.0,Marengo,Alabama,US,32.247676,-87.787962,...,243,248,255,260,264,266,273,277,286,299
48,84001087,US,USA,840,1087.0,Macon,Alabama,US,32.387599,-85.692677,...,147,150,153,159,160,161,165,166,170,177


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-30,38045,937,0,79228,20777,231418,32698,46514,11474,49,...,43509,163060,22217,1208,81,62787,32824,2905,28659,1487
2020-07-01,38962,975,0,84105,21197,238681,33012,46572,11510,49,...,45315,172368,22716,1210,90,63203,33435,2979,29199,1514
2020-07-02,40111,1014,0,87445,22075,246550,33335,46646,11731,49,...,46890,179137,23270,1227,92,63735,34151,3053,29738,1550


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-30         530     663     322  167    204     360    604     268   
2020-07-01         545     686     323  171    214     362    604     270   
2020-07-02         553     735     333  176    218     364    607     280   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-30          576       71  ...      50      3       27        6   
2020-07-01          581       74  ...      56      4       27        6   
2020-07-02          600       79  ...      57      4       27        6   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-30             90   129   176          0       39      2  
2020-07-01             89   131   177          0       39      2  
2020-07-02             92   136   180          0       39      2  

[3 rows x 3254 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-30,950,14,0,1645,270,6082,1690,4322,509,0,...,604,2455,172,56,6,1763,1332,93,784,20
2020-07-01,972,14,0,1725,277,6169,1697,4324,509,0,...,609,2503,173,56,6,1786,1339,93,786,20
2020-07-02,985,14,0,1764,279,6265,1701,4326,510,0,...,620,2542,176,56,6,1816,1342,93,793,20


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-30          11       9       1    1      1       9     27       5   
2020-07-01          11       9       1    1      1       9     27       5   
2020-07-02          12       9       1    1      1       9     27       5   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-30           27        7  ...       0      0        0        0   
2020-07-01           27        7  ...       0      0        0        0   
2020-07-02           27        7  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-30              0     0     0         19        0      0  
2020-07-01              0     0     0         19        0      0  
2020-07-02              0     0     0         19        0      0  

[3 rows x 3254 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-30,0.008931,0.027981,0.024759,0.0,0.028986,0.0,0.036327,0.016516,0.010978,0.002426,...,0.002201,0.004292,0.034177,0.054611,0.0,0.111213,0.0,0.026596,0.016582,0.029617
2020-07-01,0.010122,0.017751,0.026246,0.0,0.024648,0.0,0.041330,0.020476,0.010227,0.006023,...,0.002648,0.007479,0.032694,0.039438,0.0,0.135914,0.0,0.027634,0.023839,0.023689
2020-07-02,0.005842,0.031783,0.026976,0.0,0.082474,0.0,0.040835,0.022751,0.008124,0.003805,...,-0.094364,0.004242,0.033823,0.034807,0.0,0.116751,0.0,0.026050,0.000000,0.019835


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-30,0.017735,0.068966,0.007735,0.0,0.181818,0.0,0.021094,0.023095,0.0,0.002845,...,0.003573,0.000000,0.130435,0.062500,NaN,0.600,0.0,0.026316,0.090909,0.0
2020-07-01,0.037534,0.048387,0.008772,0.0,0.153846,0.0,0.033665,0.022573,0.0,0.000000,...,0.004017,0.037037,0.000000,0.058824,NaN,0.000,0.0,0.019231,0.250000,0.0
2020-07-02,0.042636,0.061538,0.008696,0.0,0.133333,0.0,0.025167,0.013245,0.0,0.000000,...,0.002023,0.000000,0.038462,0.055556,NaN,0.125,0.0,0.022013,0.000000,0.0


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-30,0.014138,0.014604,0.023051,0.000000,0.000000,0.000000,0.042104,0.056479,0.000426,0.003532,...,0.001462,0.002433,0.033843,0.000000,0.000000,0.008949,0.0,0.129630,0.013730,0.065789
2020-07-01,0.107565,0.039068,0.014449,0.000000,0.043011,0.000000,0.045942,0.036660,0.007102,0.000789,...,0.001460,0.001214,0.029039,0.000000,0.002985,0.019956,0.0,0.032787,0.014296,0.024691
2020-07-02,0.024919,0.028364,0.030080,0.001252,0.000000,0.045455,0.049740,0.032480,0.005642,0.001395,...,0.000729,0.003636,0.031982,0.273499,0.011905,0.000000,0.0,0.017857,0.000000,0.042169


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-30,0.023403,0.039956,NaN,0.062821,0.025670,0.034751,0.006278,0.003279,0.008615,0.0,...,0.028655,0.040547,0.025526,0.000000,0.000000,0.009616,0.017704,0.012195,0.021420,0.025517
2020-07-01,0.024103,0.040555,NaN,0.061557,0.020215,0.031385,0.009603,0.001247,0.003138,0.0,...,0.041509,0.057083,0.022460,0.001656,0.111111,0.006626,0.018614,0.025473,0.018842,0.018157
2020-07-02,0.029490,0.040000,NaN,0.039712,0.041421,0.032969,0.009784,0.001589,0.019201,0.0,...,0.034757,0.039271,0.024388,0.014050,0.022222,0.008417,0.021415,0.024841,0.018460,0.023778


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-06-30      0.017274  0.059105  0.009404  0.030864  0.040816  0.000000   
2020-07-01      0.028302  0.034691  0.003106  0.023952  0.049020  0.005556   
2020-07-02      0.014679  0.071429  0.030960  0.029240  0.018692  0.005525   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-06-30      0.003322  0.026820  0.008757  0.014286  ...  0.250000   
2020-07-01      0.000000  0.007463  0.008681  0.042254  ...  0.120000   
2020-07-02      0.004967  0.037037  0.032702  0.067568  ...  0.017857   

Province_State                                                              \
Admin2            Platte  Sheridan Sublette Sweetwater     Teton     Uinta   
2020-06-30      0.500000  0.038462      0.0   0.046512  0.007812  0.011494   
2020-07-01      0.333333  0.000000      0.0  -0.011111  0.015504  0.005682   
2020-07-02      0.000000  0.000000      0.0   0.033708  0.038168  0.016949   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-06-30            NaN      0.0    0.0  
2020-07-01            NaN      0.0    0.0  
2020-07-02            NaN      0.0    0.0  

[3 rows x 3254 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-30,0.022605,0.0,NaN,0.029412,0.018868,0.016547,0.005354,0.000463,0.003945,NaN,...,0.020270,0.016142,0.023810,0.0,0.0,0.013218,0.009091,0.0,0.009009,0.0
2020-07-01,0.023158,0.0,NaN,0.048632,0.025926,0.014305,0.004142,0.000463,0.000000,NaN,...,0.008278,0.019552,0.005814,0.0,0.0,0.013046,0.005255,0.0,0.002551,0.0
2020-07-02,0.013374,0.0,NaN,0.022609,0.007220,0.015562,0.002357,0.000463,0.001965,NaN,...,0.018062,0.015581,0.017341,0.0,0.0,0.016797,0.002240,0.0,0.008906,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                     \
Admin2           Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-30      0.000000     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2020-07-01      0.000000     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2020-07-02      0.090909     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-30          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-07-01          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-07-02          0.0      0.0  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-30            NaN   NaN   NaN        0.0      NaN    NaN  
2020-07-01            NaN   NaN   NaN        0.0      NaN    NaN  
2020-07-02            NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3254 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-30,0.009133,0.028326,0.023390,1.811883e-07,0.042325,0.031130,0.038706,0.020815,0.009875,0.003061,...,0.002548,0.004670,0.034295,0.049757,0.000518,0.109403,0.000217,0.019721,0.013170,0.020066
2020-07-01,0.009627,0.023039,0.024818,9.059415e-08,0.033486,0.015565,0.040018,0.020646,0.010051,0.004542,...,0.002598,0.006074,0.033495,0.044597,0.000259,0.122659,0.000109,0.023678,0.018504,0.021877
2020-07-02,0.007735,0.027411,0.025897,4.529707e-08,0.057980,0.007782,0.040427,0.021698,0.009088,0.004173,...,-0.045883,0.005158,0.033659,0.039702,0.000129,0.119705,0.000054,0.024864,0.009252,0.020856


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-30,0.019817,0.057722,0.007857,5.983839e-07,0.103716,6.669144e-24,0.026001,0.020853,0.000114,0.002448,...,0.002422,0.005142,0.089903,0.064327,NaN,0.383333,0.0,0.019819,0.059102,0.042155
2020-07-01,0.028675,0.053055,0.008315,2.991919e-07,0.128781,3.334572e-24,0.029833,0.021713,0.000057,0.001224,...,0.003219,0.021089,0.044952,0.061575,NaN,0.191667,0.0,0.019525,0.154551,0.021077
2020-07-02,0.035655,0.057297,0.008505,1.495960e-07,0.131057,1.667286e-24,0.027500,0.017479,0.000029,0.000612,...,0.002621,0.010545,0.041707,0.058565,NaN,0.158333,0.0,0.020769,0.077275,0.010539


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-30,0.060015,0.024849,0.023798,0.000094,0.038022,0.000003,0.051641,0.041673,0.001919,0.002486,...,0.001861,0.002500,0.031431,0.031488,0.003903,0.004927,1.271566e-06,0.074655,0.015166,0.063309
2020-07-01,0.083790,0.031958,0.019124,0.000047,0.040516,0.000002,0.048791,0.039167,0.004511,0.001638,...,0.001660,0.001857,0.030235,0.015744,0.003444,0.012441,6.357829e-07,0.053721,0.014731,0.044000
2020-07-02,0.054354,0.030161,0.024602,0.000649,0.020258,0.022728,0.049265,0.035823,0.005076,0.001516,...,0.001195,0.002747,0.031109,0.144622,0.007674,0.006221,3.178914e-07,0.035789,0.007366,0.043084


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-30,0.028722,0.033160,NaN,0.046712,0.025953,0.032582,0.006863,0.002508,0.009889,4.196069e-33,...,0.030546,0.040126,0.026089,0.001995,0.001254,0.008917,0.016301,0.013901,0.018022,0.023782
2020-07-01,0.026413,0.036858,NaN,0.054134,0.023084,0.031984,0.008233,0.001878,0.006513,2.098034e-33,...,0.036027,0.048604,0.024274,0.001825,0.056183,0.007771,0.017458,0.019687,0.018432,0.020970
2020-07-02,0.027951,0.038429,NaN,0.046923,0.032252,0.032476,0.009009,0.001733,0.012857,1.049017e-33,...,0.035392,0.043938,0.024331,0.007937,0.039202,0.008094,0.019436,0.022264,0.018446,0.022374


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-06-30      0.024671  0.072502  0.011510  0.026633  0.038853  0.006760   
2020-07-01      0.026487  0.053597  0.007308  0.025293  0.043936  0.006158   
2020-07-02      0.020583  0.062513  0.019134  0.027266  0.031314  0.005841   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-06-30      0.004370  0.042771  0.016573  0.024398  ...  0.195428   
2020-07-01      0.002185  0.025117  0.012627  0.033326  ...  0.157714   
2020-07-02      0.003576  0.031077  0.022664  0.050447  ...  0.087786   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-06-30      0.251953  0.034955  0.013029   0.044008  0.019329  0.017696   
2020-07-01      0.292643  0.017478  0.006514   0.016448  0.017416  0.011689   
2020-07-02      0.146322  0.008739  0.003257   0.025078  0.027792  0.014319   

Province_State                                   
Admin2         Unassigned      Washakie  Weston  
2020-06-30            NaN  4.356958e-10  0.2500  
2020-07-01            NaN  2.178479e-10  0.1250  
2020-07-02            NaN  1.089239e-10  0.0625  

[3 rows x 3254 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-30,0.015695,0.005209,NaN,0.019941,0.016810,0.012221,0.003754,0.000809,0.002676,NaN,...,0.015499,0.012188,0.014408,2.219461e-06,2.886626e-16,0.009979,0.007022,0.000822,0.005705,0.000027
2020-07-01,0.019426,0.002604,NaN,0.034287,0.021368,0.013263,0.003948,0.000636,0.001338,NaN,...,0.011889,0.015870,0.010111,1.109731e-06,1.443313e-16,0.011513,0.006139,0.000411,0.004128,0.000014
2020-07-02,0.016400,0.001302,NaN,0.028448,0.014294,0.014412,0.003152,0.000549,0.001651,NaN,...,0.014976,0.015726,0.013726,5.548653e-07,7.216564e-17,0.014155,0.004190,0.000206,0.006517,0.000007


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga       Baldwin Barbour Bibb Blount   Bullock   Butler   
2020-06-30      0.008293  1.144331e-13     0.0  0.0    0.0 -0.024101  0.00016   
2020-07-01      0.004147  5.721655e-14     0.0  0.0    0.0 -0.012051  0.00008   
2020-07-02      0.047528  2.860827e-14     0.0  0.0    0.0 -0.006025  0.00004   

Province_State                                    ... Wyoming                  \
Admin2           Calhoun      Chambers  Cherokee  ...    Park Platte Sheridan   
2020-06-30      0.000987  1.341517e-06  0.000700  ...     NaN    NaN      NaN   
2020-07-01      0.000493  6.707583e-07  0.000350  ...     NaN    NaN      NaN   
2020-07-02      0.000247  3.353791e-07  0.000175  ...     NaN    NaN      NaN   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-30          NaN        NaN   NaN   NaN   0.000029      NaN    NaN  
2020-07-01          NaN        NaN   NaN   NaN   0.000014      NaN    NaN  
2020-07-02          NaN        NaN   NaN   NaN   0.000007      NaN    NaN  

[3 rows x 3254 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200702,AK,1017,114892.0,NaN,18.0,NaN,NaN,NaN,1.0,...,115909,0,0,5ceb0f7088d68f2109723045dc01361a8be8f4aa,0,0,0,0,0,NaN
1,20200702,AL,40111,379617.0,NaN,843.0,2835.0,NaN,826.0,NaN,...,419728,13,32,bc01418e5a22f60d16bd176bbb53e937cef2dc5f,0,0,0,0,0,NaN
2,20200702,AR,22075,301912.0,NaN,272.0,1477.0,NaN,NaN,72.0,...,323987,2,29,2413f6ff37227446a9c86921de099b46c163315f,0,0,0,0,0,NaN
3,20200702,AS,0,696.0,NaN,NaN,NaN,NaN,NaN,NaN,...,696,0,0,2f4ba1329beda94564e2d27052e2b97d2e3c99ca,0,0,0,0,0,NaN
4,20200702,AZ,87425,473414.0,NaN,2938.0,4916.0,723.0,NaN,488.0,...,560839,37,79,4aaef59a1ee4088ad1086a7ac6f469b856ae402b,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200702,AK,1017.0,114892.0,NaN,18.0,NaN,NaN,NaN,1.0,...,115909,0,0,5ceb0f7088d68f2109723045dc01361a8be8f4aa,0,0,0,0,0,NaN
1,20200702,AL,40111.0,379617.0,NaN,843.0,2835.0,NaN,826.0,NaN,...,419728,13,32,bc01418e5a22f60d16bd176bbb53e937cef2dc5f,0,0,0,0,0,NaN
2,20200702,AR,22075.0,301912.0,NaN,272.0,1477.0,NaN,NaN,72.0,...,323987,2,29,2413f6ff37227446a9c86921de099b46c163315f,0,0,0,0,0,NaN
3,20200702,AS,0.0,696.0,NaN,NaN,NaN,NaN,NaN,NaN,...,696,0,0,2f4ba1329beda94564e2d27052e2b97d2e3c99ca,0,0,0,0,0,NaN
4,20200702,AZ,87425.0,473414.0,NaN,2938.0,4916.0,723.0,NaN,488.0,...,560839,37,79,4aaef59a1ee4088ad1086a7ac6f469b856ae402b,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-02,AK,1017,114892.0,NaN,18.0,NaN,NaN,NaN,1.0,NaN,...,115909,0,0,5ceb0f7088d68f2109723045dc01361a8be8f4aa,0,0,0,0,0,NaN
2020-07-02,AL,40111,379617.0,NaN,843.0,2835.0,NaN,826.0,NaN,468.0,...,419728,13,32,bc01418e5a22f60d16bd176bbb53e937cef2dc5f,0,0,0,0,0,NaN
2020-07-02,AR,22075,301912.0,NaN,272.0,1477.0,NaN,NaN,72.0,231.0,...,323987,2,29,2413f6ff37227446a9c86921de099b46c163315f,0,0,0,0,0,NaN
2020-07-02,AS,0,696.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,696,0,0,2f4ba1329beda94564e2d27052e2b97d2e3c99ca,0,0,0,0,0,NaN
2020-07-02,AZ,87425,473414.0,NaN,2938.0,4916.0,723.0,NaN,488.0,NaN,...,560839,37,79,4aaef59a1ee4088ad1086a7ac6f469b856ae402b,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-02,AK,1017.0,114892.0,NaN,18.0,NaN,NaN,NaN,1.0,NaN,...,115909,0,0,5ceb0f7088d68f2109723045dc01361a8be8f4aa,0,0,0,0,0,NaN
2020-07-02,AL,40111.0,379617.0,NaN,843.0,2835.0,NaN,826.0,NaN,468.0,...,419728,13,32,bc01418e5a22f60d16bd176bbb53e937cef2dc5f,0,0,0,0,0,NaN
2020-07-02,AR,22075.0,301912.0,NaN,272.0,1477.0,NaN,NaN,72.0,231.0,...,323987,2,29,2413f6ff37227446a9c86921de099b46c163315f,0,0,0,0,0,NaN
2020-07-02,AS,0.0,696.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,696,0,0,2f4ba1329beda94564e2d27052e2b97d2e3c99ca,0,0,0,0,0,NaN
2020-07-02,AZ,87425.0,473414.0,NaN,2938.0,4916.0,723.0,NaN,488.0,NaN,...,560839,37,79,4aaef59a1ee4088ad1086a7ac6f469b856ae402b,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE